#  Выбор методов решения ОДУ

Хотя алгоритмы по умолчанию, наряду с `alg_hints = [:stiff]`, в большинстве случаев подойдут, бывают ситуации, когда вам может потребоваться усилить контроль. Цель этой части руководства - познакомить вас с некоторыми из наиболее широко используемых алгоритмов. На соответствующей странице документации всё расписано подробней [ODE Solvers](http://docs.juliadiffeq.org/latest/solvers/ode_solve.html).

## Диагностика жесткости

Один из ключевых моментов, который следует знать при выборе алгоритма, заключается в том, является ли ваша проблема жесткой. Давайте возьмем, к примеру, управляемое уравнение Ван дер Поля:

In [ ]:
using DifferentialEquations, ParameterizedFunctions

In [ ]:
using LinearAlgebra

In [ ]:
van! = @ode_def VanDerPol begin
  dy = μ*((1-x^2)*y - x)
  dx = 1*y
end μ

prob = ODEProblem(van!,[0.0,2.0],(0.0,6.3),1e6)

# после обновлений, эта система легко решается - 
# задам уравнения для магнитных моментов
H0 = 0.5e-11
H1 = 2e-11
ω = 5e1
mb = 9.27e-24
h = 1.054e-34
γ = 2*mb/h
H(t) = [ H1*cos(ω*t), H1*sin(ω*t), H0 ]
syst!(μ,p,t) = γ*cross(μ, H(t) ) # ODE system

# prob = ODEProblem(syst!, [0.5;0;0.9], (0.0,6.3))


Одним из факторов, который должен предупредить вас о том, что эта модель может быть жесткой, является тот факт, что параметр равен «1e6»: большие параметры обычно означают жесткие модели. Если мы попытаемся решить это с помощью метода по умолчанию:

In [ ]:
sol = solve(prob) # с этой тоже справилась... сами придумывайте жёсткую систему

Из-за жесткости может быть достигнут предел итератора, решатель может потерять стабильность (взорван до бесконечности) или  dt модет стать слишком маленьким. Если это произойдет, первое, что нужно сделать, это проверить правильность вашей модели. Вполне возможно, что вы допустили ошибку, которая приводит к нестабильной работе модели! 

Если проблема не в модели, то причиной может быть жесткость. Таким образом, мы можем намекнуть решателю использовать соответствующий метод:

In [ ]:
sol = solve(prob,alg_hints = [:stiff]) # долго
# ахах, теперь предел итератора - яж говорил, задания Лебедева это что-то с чем-то

Другой способ выявить жесткость - это посмотреть на решение.

In [ ]:
using Plots; gr()

In [ ]:
#sol = solve(prob,alg_hints = [:stiff],reltol=1e-6)
plot(sol,denseplot=false)

Давайте увеличим ось Y, чтобы увидеть, что происходит:

In [ ]:
plot(sol,ylims = (-10.0,10.0))

Обратите внимание на то, как происходят некоторые экстремальные вертикальные сдвиги. Эти вертикальные сдвиги являются местами, где производный член очень велик, и это свидетельствует о жесткости. Это спецефичный пример, чтобы подчеркнуть поведение, но эту общую идею можно перенести на вашу проблему. Если вы сомневаетесь, просто попробуйте синхронизацию, используя как жесткий, так и нежесткий решатель, и посмотрите, какой из них более эффективен. 

Давайте попробуем BenchmarkTools, пакет, который позволяет нам относительно надежно замерять производительность блоков кода.

In [ ]:
using BenchmarkTools

Давайте сравним производительность не жестких и жестких решателей по уравнению Лоренца, которое мы видели в последнем блокноте 'ODEIntroduction'. 

Во-первых, давайте возьмем всю нашу информацию об уравнении Лоренца из последней записной книжки.

In [ ]:
function lorenz!(du,u,p,t)
    σ,ρ,β = p
    du[1] = σ*(u[2]-u[1])
    du[2] = u[1]*(ρ-u[3]) - u[2]
    du[3] = u[1]*u[2] - β*u[3]    
end
u0 = [1.0,0.0,0.0]
p = (10,28,8/3)
tspan = (0.0,100.0)
prob = ODEProblem(lorenz!,u0,tspan,p)

А теперь давайте воспользуемся макросом `@btime` из инструментов тестирования, чтобы сравнить использование не жестких и жестких решателей в этой проблеме.

In [ ]:
@btime solve(prob); # можете пока поставить чайник кипятиться

In [ ]:
@btime solve(prob,alg_hints = [:stiff]);

В этом конкретном случае мы видим, что нежесткие решатели гораздо быстрее приводят нас к решению.

## Рекомендуемые методы

При выборе метода общие правила таковы: 

- более высокий порядок (порядок метода) более эффективен при более низких погрешностях, более низкий порядок более эффективен при более высоких погрешностях 
- адаптивность необходима в большинстве реальных сценариев 
- методы Рунге-Кутты делают хорошо с не жесткими уравнениями, методы Розенброка хорошо работают с небольшими жесткими уравнениями, методы BDF хорошо работают с большими жесткими уравнениями 

Хотя из этого правила всегда есть исключения, это хорошие руководящие принципы. Исходя из этого, простой способ выбора методов: 

- по умолчанию `Tsit5()`, нежесткий метод Рунге-Кутты порядка 5 
- если вы используете низкие погрешности (`1e-8`), попробуйте `Vern7()` или `Vern9()` 
- если вы используете высокие допуски, попробуйте `BS3()` 
- если проблема жесткая, попробуйте `Rosenbrock23()`, `Rodas4()` или `CVODE_BDF() `

(Это упрощенная версия алгоритма выбора по умолчанию)

## Сравнение с другим программным обеспечением

Если вы знакомы с MATLAB, SciPy или DESolve языка R, вот быстрый перевод, чтобы передать ваши знания.

- ode23 –> BS3()
- ode45 / dopri5 -> DP5(), хотя в большинстве случаев Tsit5() более эффективен 
- ode23s -> Rosenbrock23(), хотя в большинстве случаев Rodas4() более эффективен 
- ode113 -> CVODE_Adams(), хотя во многих случаях Vern7() более эффективен 
- dop853 -> DP8(), хотя в большинстве случаев Vern7() более эффективен 
- ode15s / vode -> CVODE_BDF(), хотя во многих случаях Rodas4() или radau() более эффективность 
- ode23t -> Trapezoid() для эффективности и GenericTrapezoid() для надежности 
- ode23tb -> TRBDF2 
- lsoda -> lsoda() (требуется Pkg.add('LSODA'); с использованием LSODA) 
- ode15i -> IDA(), хотя во многих случаях Rodas4() может обрабатывать DAE и значительно более эффективен